# Quant Option Pricer Demo

Interactive exploration of Monte Carlo option pricing, Greeks, and risk metrics.

## Features
- Real-time volatility estimation from Yahoo Finance
- Black-Scholes vs Monte Carlo pricing
- Delta and Vega Greeks (analytic & MC)
- Value-at-Risk (VaR) and Conditional VaR (CVaR)
- Interactive parameter adjustment

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML

from quant_option import (
    download_log_returns, annualized_vol,
    black_scholes, bs_delta, bs_vega,
    monte_carlo_price, compute_var_cvar,
    simulate_gbm_paths
)

## 1. Data Download and Volatility Estimation

In [ ]:
# Download historical data and estimate volatility
ticker = 'AAPL'
returns = download_log_returns(ticker)
sigma = annualized_vol(returns)
print(f"Estimated annualized volatility for {ticker}: {sigma:.2%}")

# Plot returns
plt.figure(figsize=(10, 6))
returns.plot(title=f'{ticker} Daily Log-Returns')
plt.xlabel('Date')
plt.ylabel('Log-Return')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Interactive Option Pricing

In [ ]:
# Create interactive widgets
S0_widget = widgets.FloatSlider(value=150, min=50, max=300, step=5, description='Spot (S₀)')
K_widget = widgets.FloatSlider(value=150, min=50, max=300, step=5, description='Strike (K)')
r_widget = widgets.FloatSlider(value=0.01, min=0, max=0.1, step=0.001, description='Rate (r)')
T_widget = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Time (T)')
paths_widget = widgets.IntSlider(value=10000, min=1000, max=100000, step=1000, description='MC Paths')
option_type_widget = widgets.Dropdown(
    options=['call', 'put'], 
    value='call', 
    description='Option Type'
)

# Layout
ui = widgets.VBox([
    widgets.HTML(value="<h3>Option Parameters</h3>"),
    S0_widget, K_widget, r_widget, T_widget, paths_widget, option_type_widget
])

output = widgets.Output()

def update_pricing(change):
    with output:
        output.clear_output(wait=True)
        
        # Get current values
        S0 = S0_widget.value
        K = K_widget.value
        r = r_widget.value
        T = T_widget.value
        n_paths = paths_widget.value
        option_type = option_type_widget.value
        
        # Calculate prices
        bs_price = black_scholes(S0, K, r, sigma, T, option_type)
        mc_price = monte_carlo_price(S0, K, r, sigma, T, 252, n_paths, 42, option_type)
        
        # Calculate Greeks
        bs_delta_val = bs_delta(S0, K, r, sigma, T, option_type)
        bs_vega_val = bs_vega(S0, K, r, sigma, T)
        
        # Display results
        html_output = f"""
        <div style='background-color: #f0f0f0; padding: 15px; border-radius: 5px;'>
        <h4>Pricing Results ({option_type.title()} Option)</h4>
        <p><strong>Black-Scholes Price:</strong> ${bs_price:.4f}</p>
        <p><strong>Monte Carlo Price:</strong> ${mc_price:.4f} ({n_paths:,} paths)</p>
        <p><strong>Difference:</strong> ${abs(bs_price - mc_price):.4f}</p>
        <hr>
        <h4>Greeks (Analytic)</h4>
        <p><strong>Delta:</strong> {bs_delta_val:.4f}</p>
        <p><strong>Vega:</strong> {bs_vega_val:.4f}</p>
        </div>
        """
        
        display(HTML(html_output))

# Connect widgets to update function
for widget in [S0_widget, K_widget, r_widget, T_widget, paths_widget, option_type_widget]:
    widget.observe(update_pricing, names='value')

# Initial calculation
update_pricing(None)

display(ui, output)

## 3. Risk Analysis: VaR and CVaR

In [ ]:
# Calculate P&L distribution and risk metrics
S0, K, r, T = 150, 150, 0.01, 1.0
n_paths = 50000
option_type = 'call'

# Get Black-Scholes price for P&L calculation
bs_price = black_scholes(S0, K, r, sigma, T, option_type)

# Simulate paths and calculate P&L
ST = simulate_gbm_paths(S0, r, sigma, T, 252, n_paths, 42)
if option_type == 'call':
    payoff = np.maximum(ST - K, 0)
else:
    payoff = np.maximum(K - ST, 0)
pnl = np.exp(-r * T) * payoff - bs_price

# Calculate VaR and CVaR
var, cvar = compute_var_cvar(pnl, alpha=0.05)

print(f"Risk Metrics (95% confidence):")
print(f"VaR:  ${var:.4f}")
print(f"CVaR: ${cvar:.4f}")

# Plot P&L histogram
plt.figure(figsize=(10, 6))
plt.hist(pnl, bins=50, color='skyblue', edgecolor='k', alpha=0.7, density=True)
plt.axvline(-var, color='red', linestyle='--', linewidth=2, label=f'VaR (5%): ${-var:.2f}')
plt.axvline(-cvar, color='purple', linestyle=':', linewidth=2, label=f'CVaR (5%): ${-cvar:.2f}')
plt.xlabel('P&L at Expiry')
plt.ylabel('Density')
plt.title(f'P&L Distribution for {option_type.title()} Option (Monte Carlo)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Convergence Analysis

In [ ]:
# Test convergence with different numbers of paths
path_counts = [1000, 5000, 10000, 50000, 100000]
mc_prices = []
bs_price = black_scholes(S0, K, r, sigma, T, option_type)

for n_paths in path_counts:
    price = monte_carlo_price(S0, K, r, sigma, T, 252, n_paths, 42, option_type)
    mc_prices.append(price)

plt.figure(figsize=(10, 6))
plt.semilogx(path_counts, mc_prices, 'o-', label='Monte Carlo', linewidth=2, markersize=8)
plt.axhline(bs_price, color='red', linestyle='--', label='Black-Scholes', linewidth=2)
plt.xlabel('Number of Paths')
plt.ylabel('Option Price')
plt.title(f'Monte Carlo Convergence to Black-Scholes ({option_type.title()} Option)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Black-Scholes price: ${bs_price:.4f}")
for i, n_paths in enumerate(path_counts):
    error = abs(mc_prices[i] - bs_price)
    print(f"{n_paths:,} paths: ${mc_prices[i]:.4f} (error: ${error:.4f})")